In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, r2_score
import altair as alt

2024-05-26 00:14:11.262704: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 00:14:11.417957: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 00:14:17.702163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Ler os arquivos CSV em Dataframes Pandas
df_performance = pd.read_csv('/home/rhudson/Documentos/PUC/Projeto em ciência de dados I/ppl-cd-pcd-sist-int-2024-1-sleepresearch-2024-1/assets/data/Student_Performance.csv')
df_stress = pd.read_csv('/home/rhudson/Documentos/PUC/Projeto em ciência de dados I/ppl-cd-pcd-sist-int-2024-1-sleepresearch-2024-1/assets/data/Student Stress Factors.csv')

In [4]:
# Criar uma nova coluna `ID do Estudante` no `df_performance`, atribuindo um ID único inteiro para cada estudante (de 1 a 10000).
df_performance['ID do Estudante'] = range(1, len(df_performance) + 1)

In [5]:
# Randomiza 520 linhas do `df_performance` e armazená-las em `df_performance_sample`.
df_performance_sample = df_performance.sample(n=520, random_state=1)


In [6]:
# Combinar `df_performance_sample` com `df_stress` com base no índice.
df_combined = pd.merge(df_performance_sample, df_stress, left_index=True, right_index=True)


In [25]:
# Filtrar o dataframe combinado `df_combined` para manter apenas as colunas `Horas de Sono`, `Qualidade do Sono` e `Índice de Desempenho`.
#df_combined = df_combined[['Sleep Hours', 'Sleep Quality', 'Performance Index']]


In [26]:
# Separar os recursos (X) e a variável alvo (y) do `df_combined`.
X = df_combined.drop('Performance Index', axis=1)
y = df_combined['Performance Index']


In [27]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


In [28]:
# Normalizar os recursos usando MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# Construir, compilar e treinar um modelo de rede neural sequencial
model = Sequential()
model.add(Dense(X.shape[1], input_dim=X.shape[1], activation='relu'))  # Input layer
model.add(Dense(64, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='linear'))  # Output layer
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)


/home/rhudson/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 2878.5632 - mean_absolute_error: 49.9137 - val_loss: 4940.3730 - val_mean_absolute_error: 65.7424
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - loss: 2877.8337 - mean_absolute_error: 49.9068 - val_loss: 4939.6694 - val_mean_absolute_error: 65.7368
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 2877.1501 - mean_absolute_error: 49.9004 - val_loss: 4938.9351 - val_mean_absolute_error: 65.7309
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - loss: 2876.4512 - mean_absolute_error: 49.8939 - val_loss: 4938.1836 - val_mean_absolute_error: 65.7250
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step - loss: 2875.7402 - mean_absolute_error: 49.8872 - val_loss: 4937.4199 - val_mean_absolute_error: 65.7189
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - loss: 2875.0195 - mean_absolute_error: 49.8804 - val_loss: 4936.6450 - val_mean_absolute_error: 65.7128
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 28

In [30]:
# Prever no conjunto de teste e calcular o erro médio absoluto
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


In [31]:
# Calcular o R²
r2 = r2_score(y_test, y_pred)



In [34]:
# Imprimir MAE e R²
print(f'Mean Absolute Error on Test Set: {mae:.2f}')
print(f'R-squared on Test Set: {r2:.2f}')


Mean Absolute Error on Test Set: 56.71
R-squared on Test Set: -13.16


In [35]:
# Criar um DataFrame para o gráfico
df_plot = pd.DataFrame({'Real Values': y_test, 'Predicted Values': y_pred.flatten()})



In [43]:
# Criar o gráfico de dispersão
scatter_plot = alt.Chart(df_plot).mark_circle(size=60).encode(
    x=alt.X('Real Values', title='Valores Reais'),
    y=alt.Y('Predicted Values', title='Valores Preditos'),
    tooltip=['Real Values', 'Predicted Values']
).properties(
    title='Valores Reais vs. Valores Preditos'
).interactive()




In [37]:
# Criar a linha x=y
linha = alt.Chart(pd.DataFrame({'x': [y.min(), y.max()], 'y': [y.min(), y.max()]})).mark_line().encode(
    x='x',
    y='y',
    color=alt.value('red')
)

In [38]:
# Combinar o gráfico de dispersão e a linha
chart = scatter_plot + linha


In [39]:
# Salvar o gráfico como um arquivo JSON
chart.save('real_vs_predicted_values.json')
